# Video Super Resolution with DeGirum PySDK
Super Resolution is the process of enhancing the quality of an image by increasing the pixel count using deep learning. This notebook applies Single Image Super Resolution (SISR) to frames in a 360p (480×360) video in 360p resolution. A model called [single-image-super-resolution-1032](https://docs.openvino.ai/2024/omz_models_model_single_image_super_resolution_1032.html), which is available in Open Model Zoo, is used in this tutorial. It is based on the research paper cited below. 

Y. Liu et al., ["An Attention-Based Approach for Single Image Super Resolution,"](https://arxiv.org/abs/1807.06779) 2018 24th International Conference on Pattern Recognition (ICPR), 2018, pp. 2777-2784, doi: 10.1109/ICPR.2018.8545760.

> **NOTE**: The Single Image Super Resolution (SISR) model used in this demo is not optimized for a video. Results may vary depending on the video.


## Prerequisites

In [ ]:
# make sure PySDK and degirum-tools is installed
!pip show degirum || pip install -q degirum
!pip show degirum-tools || pip install -q degirum-tools

## Imports and settings

In [ ]:
import cv2
import degirum as dg
import degirum_tools as dgtools

from superres_postprocessor import SuperResolutionResults

hw_location = dg.LOCAL
zoo_url = 'https://cs.degirum.com/degirum/super_resolution'

model_name = 'singleimage_superresolution_4x--480x270_float_openvino_cpu_1'
SuperResolutionResults.resize_factor = 4

video_results_path = "temp/superres_comparison.mp4"
num_frames = 200  # Maximum number of frames to read from video, set to 0 for all frames.

In [ ]:
from contextlib import contextmanager
from functools import cmp_to_key
from pathlib import Path
import urllib

from degirum_tools import environment as env

@contextmanager
def open_video_stream(video_source, max_quality): 
    """Open OpenCV video stream from camera with given identifier.

    video_source - 0-based index for local cameras
       or IP camera URL in the format "rtsp://<user>:<password>@<ip or hostname>",
       or local video file path,
       or URL to mp4 video file,
       or YouTube video URL
    max_quality - The maximum video quality for YouTube videos. The units are
       in pixels for the height of the video.
    Returns context manager yielding video stream object and closing it on exit
    """

    if env.get_test_mode() or video_source is None:
        video_source = env.get_var(env.var_VideoSource, 0)
        if isinstance(video_source, str) and video_source.isnumeric():
            video_source = int(video_source)

    if isinstance(video_source, Path):
        video_source = str(video_source)

    if isinstance(video_source, str) and urllib.parse.urlparse(
        video_source
    ).hostname in (
        "www.youtube.com",
        "youtube.com",
        "youtu.be",
    ):  # if source is YouTube video
        import pafy

        video_qualities = pafy.new(video_source).videostreams
        # Sort descending based on vertical pixel count.
        video_qualities = sorted(video_qualities, key=cmp_to_key(lambda a, b: b.dimensions[1] - a.dimensions[1]))
        
        for video in video_qualities:
            if video.dimensions[1] <= max_quality:
                video_source = video.url
                break

    stream = cv2.VideoCapture(video_source)  # type: ignore[arg-type]
    if not stream.isOpened():
        raise Exception(f"Error opening '{video_source}' video stream")
    else:
        print(f"Successfully opened video stream '{video_source}'")

    try:
        yield stream
    finally:
        stream.release()

## Run inference on a video

In [ ]:
import numpy as np

video_source = "https://www.youtube.com/watch?v=V8yS3WIkOrA"

# Change backend back to opencv for compatibility with dgstreams
zoo = dg.connect(hw_location, zoo_url, dgtools.get_token())
model = zoo.load_model(model_name, image_backend='opencv', input_image_format="RAW")

model.custom_postprocessor = SuperResolutionResults

with open_video_stream(video_source, max_quality=360) as video_stream:
    w, h, fps = dgtools.get_video_stream_properties(video_stream)

    if num_frames == 0:
        total_frames = video_stream.get(cv2.CAP_PROP_FRAME_COUNT)
    else:
        total_frames = num_frames

    with dgtools.open_video_writer(video_results_path, w * 2, h, fps) as writer:
        progress = dgtools.Progress(total_frames)

        for i, frame in enumerate(dgtools.video_source(video_stream)):
            if i == total_frames:
                break
            
            inference_result = model([frame, frame])

            # Stack super resolution result on the left and bicubic result on the right.
            h, w, c = inference_result.image_overlay.shape

            compound_image = np.zeros((h, w * 2, c), dtype=np.uint8)
            compound_image[:h,:w] = inference_result.image_overlay
            compound_image[:h, w:] = cv2.resize(inference_result.image, (w, h), cv2.INTER_CUBIC)
            writer.write(compound_image)
            progress.step()

In [ ]:
dgtools.ipython_display(video_results_path, True)